### Cálculo de las unidades equivalentes con el modelo

In [1]:
# Library import

import warnings

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

import pandas as pd
from pathlib import Path
import datetime as dt
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

delta=dt.timedelta(days=1)

In [2]:
# Get main path and other folders
s_mainpath=Path.cwd()
print(s_mainpath)
s_parentpath=s_mainpath.parent.parent.parent
print(s_parentpath)
# folderpath=s_parentpath.joinpath(r'DatosXM\Despacho\\')

# Ruta General
sRutaDesp=r'C:\Información XM\\'

sRutaPrint=s_mainpath.joinpath('Print')

c:\Alejo\Eje de Planeación\Análisis Eléctrico\ScriptsPython
c:\Alejo


In [3]:
# Función para calcular la demanda de Caribe 2 de los archivos PRONAREAS
def CalDemCar2(sFolder):
    # Demanda Caribe archivos demanda pronóstico
    pathfoder=Path('C:\AnalisisElectrico\Demanda')
    # sFolder='Oficial_Ini'
    ano=2024
    pathfoder=pathfoder.joinpath(pathfoder,sFolder)

    # Definir encabezados
    l_col=['Subarea','Per']
    l_col2=['Subarea','Per','Tipo']
    l_values=[]
    for i in range(1,8):
        l_col.append(i)
        l_col2.append(i)
        l_values.append(i)


    # stfiles=[file.lower() for file in os.listdir(pathfoder)]
    # Determinar lso archivos que están dentro de la carpeta
    stfiles=os.listdir(pathfoder)
    stfiles=sorted(stfiles)

    df_DemPA=pd.DataFrame()
    # Recorrer el nombre de cada uno de los archivos para leerlo
    for file in stfiles:
        
        mes=file[-8:-6]
        dia=file[-6:-4]
        # print(file,mes,dia)
        pathFile=pathfoder.joinpath(file)

        # Identificar si el archivo tiene formato diferente
        with open(pathFile, 'r') as file:
            # Read the first line
            first_line = file.readline()
            split_line = first_line.split(',')

        if (split_line[2]=='EN') or (split_line[2]=='POT'):

            df_Aux = pd.read_csv(pathFile, sep=',',names=l_col2,encoding="ISO-8859-1")
            df_Aux.drop(['Tipo'],axis=1,inplace=True)
            l_sub=['SubAtlantico','SubBolivar','SubGCM']
        else:
            df_Aux = pd.read_csv(pathFile, sep=',',names=l_col,encoding="ISO-8859-1")
            l_sub=['SUBATLANTICO','SUBBOLIVAR','SUBGCM']


        # Determinar la fecha inicial a partir de fecha del archivo de pronóstico
        s_FechaIni=str(ano) + '-' + str(mes) + '-' + str(dia)
        dFecha=dt.datetime.strptime(s_FechaIni,'%Y-%m-%d')
        dFechaFin=dFecha+(6*delta)

        # Hacer el recorrido de las fechas durante 7 días
        icont=1
        while dFecha<=dFechaFin:

            # Obtener los datos de cada día y asignar la fecha
            df_dia=df_Aux[['Subarea','Per',icont]]

            df_Aux1=df_dia.pivot(index=['Subarea'],columns=['Per'],values=[icont])
            df_Aux1=pd.DataFrame(df_Aux1.loc[l_sub].sum(axis=0)).T
            df_Aux1.columns = [col[1] for col in df_Aux1.columns]
            df_Aux1.reset_index(drop=True,inplace=True)
            df_Aux1['Fecha']=dFecha
            df_Aux1.set_index(['Fecha'],inplace=True)

            df_DemPA=pd.concat([df_DemPA,df_Aux1])

            icont=icont+1
            dFecha=dFecha+delta

    return df_DemPA

In [4]:
# Función para asignar los días de la semana a cada fecha, si es festivo se trata como un domingo
import holidays
co_holidays = holidays.Colombia()

def typedays(row,weekday):

     if weekday==True:
          return row['Fecha'].weekday()
     else:
          if row['Fecha'] in co_holidays:
               return 3
          elif row['Fecha'].weekday()==5:
               return 2
          elif row['Fecha'].weekday()==6:
               return 3
          else:
               return 1

In [5]:
# Importar modelo
import joblib
Dic_Data_Modelo = joblib.load('DataModel.pkl')

modelo_final=Dic_Data_Modelo['Model']
scaler_X=Dic_Data_Modelo['scaler_X']
scaler_y=Dic_Data_Modelo['scaler_y']


In [13]:
df_DemPA_Sem=CalDemCar2('SemanaAnalisis')

# Procesar la información de demanda
df_Dem=df_DemPA_Sem.copy()
df_Dem.reset_index(inplace=True)

# Definir tipo de día
df_Dem['day_osf']=df_Dem.apply(lambda row: typedays(row,weekday=False),axis=1)
# Definir día de la semana
df_Dem['day_w']=df_Dem.apply(lambda row: typedays(row,weekday=True),axis=1)
# Definir los periodos en una columna
df_Dem = pd.melt(df_Dem, id_vars=['Fecha','day_osf','day_w'], var_name='Periodo', value_name='Dem')

In [14]:
df_DemPA_Sem.to_csv('DemandaCaribe2_2.csv')

In [8]:
df_DemPA_Sem

,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
Fecha,,,,,,,,,,,,,,,,,,,,,
2024-05-20,2012.485408,1961.091227,1905.555759,1854.294378,1805.588611,1724.945821,1700.637622,1764.828461,1826.045512,1878.175141,...,2207.056098,2233.946965,2129.170921,1993.943278,2218.081936,2294.560548,2340.898764,2219.428727,2203.232578,2121.343080
2024-05-21,2053.465001,2003.531028,1948.018425,1907.533717,1857.761105,1776.499684,1748.862035,1830.027571,1872.363564,1918.405245,...,2250.662573,2283.079295,2165.762443,2020.958304,2248.966557,2316.099499,2364.976038,2233.712374,2218.316052,2142.065923
2024-05-22,1979.716463,1930.189087,1872.050264,1824.870897,1778.381572,1699.081673,1663.346990,1712.028512,1762.797858,1801.161919,...,2101.683956,2124.018377,2043.592930,1922.903498,2143.002339,2214.058037,2263.895473,2146.688314,2131.605617,2072.724746
2024-05-23,1993.607776,1941.069889,1882.113605,1834.800778,1784.688230,1702.211576,1661.064213,1721.734943,1774.577796,1818.785518,...,2108.619556,2141.005119,2062.230949,1936.185740,2161.936081,2227.061023,2280.357159,2151.701987,2133.015257,2073.048393
2024-05-24,1998.184882,1942.055751,1886.502827,1837.401577,1791.406787,1710.001681,1675.945149,1731.092670,1781.747918,1829.139573,...,2113.548372,2144.334238,2063.601078,1939.960947,2160.515399,2225.465314,2276.335906,2147.004484,2143.510620,2085.588383
2024-05-25,1980.223147,1926.229972,1869.390246,1822.219932,1778.827490,1718.828848,1662.752668,1706.858458,1781.378786,1842.217173,...,2027.051903,2029.803235,1954.014894,1869.131950,2087.641419,2140.907168,2158.064316,2015.084338,2021.452803,1996.773569
2024-05-26,1981.308054,1929.761395,1879.451106,1835.507057,1791.835410,1716.066447,1647.673987,1681.920033,1723.862005,1762.447018,...,1971.189792,1971.886947,1871.569045,1783.554091,2029.828858,2094.513648,2147.699073,2047.814533,2057.033340,2026.230021


In [9]:
df_data1=df_Dem.copy()
df_Uni_Pre=df_data1[['Fecha','Periodo']]

df_data1.to_csv('Data.csv')

l_col=list(df_data1.columns)
l_col.remove('Fecha')
df_dataMod1=df_data1[l_col]
df_dataMod1


nombres = df_dataMod1.columns
Var_X = scaler_X.fit_transform(df_dataMod1)
Var_X = pd.DataFrame(Var_X, columns = nombres)

predic=modelo_final.predict(Var_X)
Uni_Pre=scaler_y.inverse_transform(predic.reshape(-1, 1))
df_Uni_Pre['Uni']= Uni_Pre.flatten()
df_Uni_Pre=df_Uni_Pre.pivot(index=['Fecha'],columns=['Periodo'],values=['Uni'])

df_Uni_Pre.to_csv(str(sRutaPrint) + r'\UniRF.csv')